In [5]:
import os 
import cv2
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.models import Sequential
from keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
     

In [6]:
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  # crawls inside folders
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count
train_dir =r"C:\food\pythonProject\image data\tomato\train"
test_dir=r"C:\food\pythonProject\image data\tomato\val"

In [7]:
 #train file image count
train_samples =get_files(train_dir)
#to get tags
num_classes=len(glob.glob(train_dir+"/*")) 
#test file image count
test_samples=get_files(test_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

10 Classes
14529 Train images
3631 Test images


In [8]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
test_datagen=ImageDataGenerator(rescale=1./255)

In [9]:
input_shape=(256,256,3)
train_generator =train_datagen.flow_from_directory(train_dir,target_size=(256,256),batch_size=32)
test_generator=test_datagen.flow_from_directory(test_dir,shuffle=True,target_size=(256,256),batch_size=32)

Found 14529 images belonging to 10 classes.
Found 3631 images belonging to 10 classes.


In [10]:
validation_generator = train_datagen.flow_from_directory(
                       test_dir, # same directory as training data
                       target_size=(256,256),
                       batch_size=32)

Found 3631 images belonging to 10 classes.


In [11]:
learning_rate = 5e-5
train_batch_size = 256
eval_batch_size = 256
num_epochs = 30
num_classes = 10  # Replace with the actual number of classes in your dataset

# Load pretrained VGG16 model
pretrained_model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3)
)

# Freeze all layers up to a specific layer
freeze_until_layer = 'block4_conv3'
for layer in pretrained_model.layers:
    if layer.name == freeze_until_layer:
        break
    layer.trainable = False

# Add custom classification head
x = Flatten()(pretrained_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model2 = Model(inputs=pretrained_model.input, outputs=predictions)

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model2.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define learning rate scheduler
def lr_scheduler(epoch, lr):
    return lr * (1 - epoch / num_epochs)

lr_callback = LearningRateScheduler(lr_scheduler)

# Train the model
history = model2.fit(
    train_generator,
    epochs=num_epochs,
    steps_per_epoch=train_generator.samples // train_batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // eval_batch_size,
    verbose=1,
    callbacks=[lr_callback]
)


Epoch 1/30
56/56 [==============================] - 713s 13s/step - loss: 1.2063 - accuracy: 0.5898 - val_loss: 0.7166 - val_accuracy: 0.7634 - lr: 5.0000e-05
Epoch 2/30
56/56 [==============================] - 689s 12s/step - loss: 0.5923 - accuracy: 0.8080 - val_loss: 0.4421 - val_accuracy: 0.8549 - lr: 4.8333e-05
Epoch 3/30
56/56 [==============================] - 679s 12s/step - loss: 0.4045 - accuracy: 0.8610 - val_loss: 0.3639 - val_accuracy: 0.8839 - lr: 4.5111e-05
Epoch 4/30
56/56 [==============================] - 633s 11s/step - loss: 0.3060 - accuracy: 0.8968 - val_loss: 0.2771 - val_accuracy: 0.9085 - lr: 4.0600e-05
Epoch 5/30
56/56 [==============================] - 620s 11s/step - loss: 0.2658 - accuracy: 0.9085 - val_loss: 0.3968 - val_accuracy: 0.8638 - lr: 3.5187e-05
Epoch 6/30
56/56 [==============================] - 558s 10s/step - loss: 0.2096 - accuracy: 0.9291 - val_loss: 0.1949 - val_accuracy: 0.9353 - lr: 2.9322e-05
Epoch 7/30
56/56 [============================

In [12]:
test_loss, test_accuracy = model2.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

114/114 [==============================] - 650s 6s/step - loss: 0.1393 - accuracy: 0.9554
Test Loss: 0.1393280029296875
Test Accuracy: 0.9553841948509216


In [13]:
model2.save("tomato.h5")